# Imports and Setup

In [1]:
import json
import os
from typing import Dict, List, Union
import numpy as np
import pandas as pd

# Helper Functions

In [2]:
def load_json(file_path: str) -> Dict:
    """Load JSON data from a file."""
    with open(file_path, 'r') as f:
        return json.load(f)

def calculate_averages(data: Dict) -> Dict[str, float]:
    """Calculate averages for nested dictionaries and lists."""
    averages = {}
    for key, value in data.items():
        if isinstance(value, list):
            if isinstance(value[0], dict):
                for subkey in value[0].keys():
                    averages[f"{key}_{subkey}"] = np.mean([item[subkey] for item in value])
            else:
                averages[key] = np.mean(value)
        elif isinstance(value, (int, float)):
            averages[key] = value
    return averages

def process_scenario(llm_path: str, scenario: str, agent_types: List[str]) -> pd.DataFrame:
    """Process data for a specific scenario and agent types."""
    results = []
    for agent_type in agent_types:
        file_path = os.path.join(llm_path, agent_type, scenario)
        if os.path.exists(file_path):
            data = load_json(file_path)
            if scenario == 'assignment_test.json':
                avg_a = calculate_averages(data['claim_A'])
                avg_b = calculate_averages(data['claim_B'])
                avg = {k: (avg_a.get(k, 0) + avg_b.get(k, 0)) / 2 for k in set(avg_a) | set(avg_b)}
            else:
                avg = calculate_averages(data)
            avg['Agent'] = agent_type
            results.append(avg)
    return pd.DataFrame(results)

def highlight_max(s: pd.Series) -> List[str]:
    """Highlight the maximum value in a series."""
    is_numeric = pd.to_numeric(s, errors='coerce').notnull()
    if is_numeric.any():
        max_value = s[is_numeric].max()
        return ['background-color: #4CAF50' if v == max_value else '' for v in s]
    else:
        return ['' for _ in s]

def format_table(df: pd.DataFrame, title: str) -> pd.io.formats.style.Styler:
    """Format and style the DataFrame for better readability."""
    # Ensure 'Agent' is a column, not the index
    if 'Agent' not in df.columns:
        df = df.reset_index()
    
    # Melt the DataFrame to get the metrics as rows
    df_melted = df.melt(id_vars=['Agent'], var_name='Metric', value_name='Value')
    
    # Pivot the melted DataFrame
    df_pivot = df_melted.pivot(index='Metric', columns='Agent', values='Value')
    
    # Rename columns for better readability
    column_rename = {
        'base': 'Base',
        'exp': 'EXP',
        'coh': 'COH',
        'base_exp': 'Base+EXP',
        'base_exp_coh': 'Base+EXP+COH'
    }
    df_pivot = df_pivot.rename(columns=column_rename)
    
    # Reorder columns
    column_order = ['Base', 'EXP', 'COH', 'Base+EXP', 'Base+EXP+COH']
    df_pivot = df_pivot[column_order]
    
    # Calculate Relative Gain
    df_pivot['Relative Gain'] = df_pivot.apply(
        lambda row: f"{(row.max() / row['Base'] - 1) * 100:.2f}%" if row['Base'] != 0 else "N/A",
        axis=1
    )
    
    # Rename index for better readability
    index_rename = {
        'bertscore_F1': 'BERTScore (F1)',
        'bertscore_Precision': 'BERTScore (Precision)',
        'bertscore_Recall': 'BERTScore (Recall)',
        'fluency': 'Fluency',
        'coherence': 'Coherence',
        'explanation_accuracy': 'Explanation Accuracy',
        'explanation_completeness': 'Explanation Completeness',
        'claim_accuracy_score': 'Claim Accuracy',
        'claim_support': 'Claim Support',
        'fact_verification': 'Fact Verification'
    }
    df_pivot.index = df_pivot.index.map(lambda x: index_rename.get(x, x))

    # Reorder for better Readability
    reorder_schema = [
        'BERTScore (F1)',
        'BERTScore (Precision)',
        'BERTScore (Recall)',
        'Fluency',
        'Coherence',
        'Explanation Accuracy',
        'Explanation Completeness',
        'Claim Accuracy',
        'Claim Support',
        'Fact Verification'
    ]
    df_pivot = df_pivot.reindex(reorder_schema)
    
    # Apply highlighting only to numeric columns
    numeric_columns = df_pivot.columns[:-1]  # All columns except 'Relative Gain'
    styled_df = df_pivot.style.apply(highlight_max, axis=1, subset=numeric_columns)
    
    # Format numeric columns
    styled_df = styled_df.format({col: "{:.4f}" for col in numeric_columns})
    
    # Adjust title based on the requested changes, focusing on scenarios
    scenario_adjustments = {
        "Baseline": "Baseline Test",
        "Missing Evidence": "Missing Evidence Test",
        "Wrong Evidence": "Wrong Evidence Test",
        "Mixed": "Mixed Evidence Test",
        "Full Data Noid": "R4C Test"
    }
    
    for old_scenario, new_scenario in scenario_adjustments.items():
        if old_scenario in title:
            title = title.replace(old_scenario, new_scenario)
            break
    
    styled_df = styled_df.set_caption(title)
    styled_df = styled_df.set_table_styles([
        {'selector': 'caption', 'props': [('font-size', '1.2em'), ('font-weight', 'bold')]},
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]},
        {'selector': '', 'props': [('border', '1px solid black'), ('border-collapse', 'collapse')]},
        {'selector': 'th, td', 'props': [('padding', '5px')]},
    ])
    
    return styled_df

def process_llm(base_path: str, llm: str, scenarios: List[str], agent_types: List[str]) -> None:
    """Process and display results for a specific LLM."""
    llm_path = os.path.join(base_path, llm)
    llm_name = llm.split('/')[-1].upper()
    
    print(f"Results for {llm_name}")
    print("=" * 50)
    
    for scenario in scenarios:
        df = process_scenario(llm_path, scenario, agent_types)
        if df.empty:
            print(f"No data found for {llm_name} - {scenario}")
            continue
        
        title = f"{llm_name} - {scenario.replace('.json', '').replace('_', ' ').title()}"
        styled_df = format_table(df, title)
        display(styled_df)
        print("\n")

# Generate Tables

In [3]:
# Define paths and parameters
base_path = '../results'
scenarios = ['baseline.json', 'missing_evidence.json', 'wrong_evidence.json', 'mixed.json', 'selection_test.json', 'assignment_test.json', 'full_data_noid.json']
agent_types = ['base', 'exp', 'coh', 'base_exp', 'base_exp_coh']

# GPT-4o Tables

In [4]:
print("Processing GPT-4o")
process_llm(base_path, 'OpenAI/gpt-4o', scenarios, agent_types)

Processing GPT-4o
Results for GPT-4O


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5899,0.5893,0.5950,0.5877,0.5899,0.87%
BERTScore (Precision),0.5740,0.5836,0.5836,0.5775,0.5737,1.67%
BERTScore (Recall),0.6093,0.5984,0.6099,0.6015,0.6101,0.12%
Fluency,0.8077,0.7955,0.7979,0.7885,0.8126,0.61%
Coherence,0.9005,0.7330,0.8655,0.9503,0.8948,5.54%
Explanation Accuracy,0.8356,0.8223,0.8387,0.8391,0.8326,0.42%
Explanation Completeness,0.6669,0.6488,0.6656,0.6653,0.6724,0.82%
Claim Accuracy,0.8515,0.8506,0.8464,0.8515,0.8580,0.76%
Claim Support,1.7955,1.8075,1.9917,2.0633,1.7745,14.91%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5946,0.5886,0.5946,0.5878,0.5917,0.00%
BERTScore (Precision),0.5792,0.5767,0.5861,0.5722,0.5807,1.19%
BERTScore (Recall),0.6131,0.6037,0.6059,0.6076,0.6072,0.00%
Fluency,0.7983,0.8066,0.7996,0.8189,0.7740,2.58%
Coherence,0.8515,0.8253,0.7953,0.7868,0.8089,0.00%
Explanation Accuracy,0.8313,0.8308,0.8324,0.8373,0.8405,1.11%
Explanation Completeness,0.6622,0.6634,0.6602,0.6671,0.6771,2.25%
Claim Accuracy,0.8483,0.8477,0.8510,0.8434,0.8490,0.31%
Claim Support,1.8774,1.7140,1.8164,1.7415,2.0106,7.09%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5933,0.5872,0.5920,0.5904,0.5948,0.24%
BERTScore (Precision),0.5847,0.5769,0.5797,0.5820,0.5775,0.00%
BERTScore (Recall),0.6060,0.6008,0.6077,0.6020,0.6153,1.54%
Fluency,0.7825,0.8039,0.7961,0.8116,0.8072,3.73%
Coherence,0.9073,0.7132,0.7590,0.8973,0.7814,0.00%
Explanation Accuracy,0.8319,0.8401,0.8276,0.8290,0.8416,1.17%
Explanation Completeness,0.6588,0.6696,0.6631,0.6614,0.6695,1.63%
Claim Accuracy,0.8499,0.8531,0.8521,0.8599,0.8509,1.18%
Claim Support,1.7982,1.6856,2.1460,1.8550,1.6562,19.34%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5903,0.5866,0.5949,0.5813,0.5894,0.77%
BERTScore (Precision),0.5727,0.5730,0.5872,0.5684,0.5759,2.54%
BERTScore (Recall),0.6113,0.6036,0.6057,0.5967,0.6072,0.00%
Fluency,0.8042,0.7971,0.7890,0.7988,0.8120,0.98%
Coherence,0.7362,0.7257,0.8873,0.7138,0.8148,20.52%
Explanation Accuracy,0.8308,0.8347,0.8406,0.8184,0.8304,1.18%
Explanation Completeness,0.6645,0.6714,0.6668,0.6544,0.6754,1.65%
Claim Accuracy,0.8497,0.8555,0.8562,0.8548,0.8533,0.77%
Claim Support,1.7643,2.0178,1.9900,1.7381,1.9203,14.37%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6019,0.5826,0.5864,0.5789,0.5921,0.00%
BERTScore (Precision),0.5985,0.5779,0.5910,0.5775,0.5912,0.00%
BERTScore (Recall),0.6087,0.5894,0.5839,0.5822,0.5947,0.00%
Fluency,0.7336,0.6420,0.6870,0.6736,0.6177,0.00%
Coherence,0.9258,0.8155,0.8635,0.8117,0.9306,0.53%
Explanation Accuracy,0.8330,0.8096,0.7925,0.7913,0.8041,0.00%
Explanation Completeness,0.6473,0.6235,0.6275,0.6206,0.6172,0.00%
Claim Accuracy,0.8360,0.7947,0.7930,0.7896,0.7911,0.00%
Claim Support,1.8905,1.7875,1.6589,1.4952,1.7969,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6231,0.6035,0.6135,0.5950,0.6099,0.00%
BERTScore (Precision),0.6288,0.6145,0.6272,0.6074,0.6155,0.00%
BERTScore (Recall),0.6195,0.5947,0.6022,0.5863,0.6058,0.00%
Fluency,0.7833,0.7888,0.8010,0.7562,0.7951,2.27%
Coherence,0.9543,0.9099,0.8588,0.9320,0.8357,0.00%
Explanation Accuracy,0.8579,0.8495,0.8515,0.8244,0.8527,0.00%
Explanation Completeness,0.6152,0.6041,0.6050,0.5928,0.6123,0.00%
Claim Accuracy,0.8815,0.8815,0.8722,0.8429,0.8658,0.00%
Claim Support,2.1974,2.1209,2.2451,2.0689,2.1627,2.17%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6150,0.6110,0.6307,0.6020,0.6100,2.55%
BERTScore (Precision),0.5340,0.5254,0.5438,0.5207,0.5278,1.85%
BERTScore (Recall),0.7302,0.7349,0.7553,0.7185,0.7263,3.44%
Fluency,0.7899,0.7631,0.7787,0.7873,0.7700,0.00%
Coherence,0.9045,0.7179,0.7417,0.8967,0.9177,1.46%
Explanation Accuracy,0.8065,0.7981,0.8217,0.8048,0.8144,1.89%
Explanation Completeness,0.7628,0.7593,0.7955,0.7746,0.7833,4.29%
Claim Accuracy,0.7975,0.7896,0.8095,0.7922,0.8016,1.50%
Claim Support,0.6528,0.5373,0.6371,0.9701,0.7327,48.61%


# GPT-4o-mini Tables

In [5]:
print("Processing GPT-4o-mini")
process_llm(base_path, 'OpenAI/gpt-4o-mini', scenarios, agent_types)

Processing GPT-4o-mini
Results for GPT-4O-MINI


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5874,0.5790,0.5865,0.5772,0.5843,0.00%
BERTScore (Precision),0.5771,0.5794,0.5865,0.5758,0.5828,1.65%
BERTScore (Recall),0.6013,0.5806,0.5888,0.5814,0.5887,0.00%
Fluency,0.8043,0.7308,0.7796,0.7306,0.7704,0.00%
Coherence,0.8369,0.9129,0.8692,0.9572,0.8945,14.37%
Explanation Accuracy,0.8280,0.8183,0.8212,0.8299,0.8359,0.95%
Explanation Completeness,0.6428,0.6388,0.6399,0.6411,0.6437,0.15%
Claim Accuracy,0.8500,0.8611,0.8402,0.8629,0.8678,2.10%
Claim Support,2.0171,1.8286,1.6119,1.8616,2.0065,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5857,0.5830,0.5875,0.5780,0.5852,0.31%
BERTScore (Precision),0.5774,0.5815,0.5864,0.5734,0.5870,1.66%
BERTScore (Recall),0.5971,0.5877,0.5913,0.5855,0.5862,0.00%
Fluency,0.7905,0.7344,0.7899,0.7845,0.7561,0.00%
Coherence,0.8609,0.9147,0.8015,0.9445,0.8993,9.71%
Explanation Accuracy,0.8269,0.8200,0.8276,0.8235,0.8320,0.62%
Explanation Completeness,0.6583,0.6426,0.6415,0.6463,0.6516,0.00%
Claim Accuracy,0.8623,0.8513,0.8515,0.8599,0.8509,0.00%
Claim Support,1.9640,1.6375,1.6291,1.8742,1.9158,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5853,0.5779,0.5864,0.5825,0.5844,0.19%
BERTScore (Precision),0.5775,0.5789,0.5892,0.5820,0.5850,2.02%
BERTScore (Recall),0.5965,0.5787,0.5866,0.5855,0.5864,0.00%
Fluency,0.7828,0.7652,0.7730,0.7551,0.7709,0.00%
Coherence,0.8988,0.8890,0.9150,0.8574,0.9289,3.35%
Explanation Accuracy,0.8171,0.8180,0.8230,0.8255,0.8211,1.03%
Explanation Completeness,0.6436,0.6377,0.6525,0.6444,0.6340,1.39%
Claim Accuracy,0.8478,0.8582,0.8409,0.8677,0.8405,2.35%
Claim Support,2.0892,1.7562,1.5331,1.9979,1.7848,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5841,0.5803,0.5866,0.5734,0.5899,1.00%
BERTScore (Precision),0.5752,0.5856,0.5903,0.5708,0.5908,2.71%
BERTScore (Recall),0.5962,0.5778,0.5857,0.5780,0.5910,0.00%
Fluency,0.8032,0.7330,0.7627,0.7602,0.7713,0.00%
Coherence,0.9437,0.9087,0.9002,0.8626,0.8814,0.00%
Explanation Accuracy,0.8262,0.8200,0.8253,0.8141,0.8351,1.08%
Explanation Completeness,0.6444,0.6397,0.6486,0.6360,0.6440,0.64%
Claim Accuracy,0.8470,0.8558,0.8516,0.8443,0.8641,2.02%
Claim Support,2.0385,1.6233,1.4283,1.6386,1.6760,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5919,0.5784,0.5750,0.5708,0.5846,0.00%
BERTScore (Precision),0.5842,0.5761,0.5752,0.5654,0.5803,0.00%
BERTScore (Recall),0.6024,0.5833,0.5773,0.5788,0.5919,0.00%
Fluency,0.7405,0.6013,0.5950,0.6050,0.6424,0.00%
Coherence,0.8960,0.8722,0.7344,0.8189,0.8376,0.00%
Explanation Accuracy,0.8207,0.7830,0.7849,0.7603,0.7648,0.00%
Explanation Completeness,0.6336,0.6187,0.5973,0.6029,0.6027,0.00%
Claim Accuracy,0.8217,0.7934,0.8100,0.7705,0.7736,0.00%
Claim Support,1.6221,1.5669,1.5957,1.6005,1.2304,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6133,0.5912,0.5801,0.5888,0.5477,0.00%
BERTScore (Precision),0.6182,0.6023,0.6022,0.6005,0.5662,0.00%
BERTScore (Recall),0.6108,0.5820,0.5616,0.5792,0.5327,0.00%
Fluency,0.7650,0.7830,0.7791,0.7675,0.7640,2.35%
Coherence,0.8143,0.9259,0.8692,0.9173,0.8356,13.70%
Explanation Accuracy,0.8532,0.8415,0.7740,0.8331,0.6154,0.00%
Explanation Completeness,0.6000,0.5919,0.5534,0.5864,0.4369,0.00%
Claim Accuracy,0.8679,0.8756,0.8102,0.8673,0.6351,0.89%
Claim Support,2.0953,1.9363,1.8560,2.0712,1.6396,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6260,0.6081,0.6346,0.6063,0.6117,1.36%
BERTScore (Precision),0.5423,0.5218,0.5491,0.5163,0.5240,1.26%
BERTScore (Recall),0.7456,0.7362,0.7557,0.7381,0.7399,1.35%
Fluency,0.7505,0.7575,0.7587,0.7805,0.7654,4.00%
Coherence,1.0079,0.9378,0.9720,0.8780,0.9140,0.00%
Explanation Accuracy,0.8091,0.7777,0.8063,0.7858,0.7569,0.00%
Explanation Completeness,0.7690,0.7803,0.7769,0.7810,0.7631,1.56%
Claim Accuracy,0.8032,0.7740,0.8118,0.7872,0.7690,1.08%
Claim Support,0.7453,0.7999,0.6668,0.9056,0.9281,24.53%


# GPT 3.5 Turbo Tables

In [6]:
print("Processing GPT-3.5-turbo")
process_llm(base_path, 'OpenAI/gpt3.5-turbo', scenarios, agent_types)

Processing GPT-3.5-turbo
Results for GPT3.5-TURBO


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5097,0.5409,0.4091,0.5112,0.5500,7.91%
BERTScore (Precision),0.5158,0.5499,0.4191,0.5210,0.5589,8.34%
BERTScore (Recall),0.5066,0.5344,0.4012,0.5038,0.5438,7.33%
Fluency,0.6242,0.7169,0.5228,0.6770,0.7071,14.86%
Coherence,0.8110,0.6757,0.6489,0.5018,0.8753,7.92%
Explanation Accuracy,0.6946,0.7477,0.5919,0.7141,0.7887,13.56%
Explanation Completeness,0.5461,0.5993,0.4455,0.5561,0.6077,11.28%
Claim Accuracy,0.7264,0.7692,0.6225,0.7436,0.7950,9.44%
Claim Support,1.6526,1.5675,1.1790,1.6814,1.6293,1.74%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.4755,0.5371,0.4730,0.4205,0.4844,12.96%
BERTScore (Precision),0.4797,0.5436,0.4747,0.4358,0.4912,13.33%
BERTScore (Recall),0.4742,0.5328,0.4728,0.4094,0.4794,12.37%
Fluency,0.6272,0.6991,0.6167,0.5069,0.6301,11.48%
Coherence,0.7212,0.7569,0.6088,0.6222,0.7737,7.28%
Explanation Accuracy,0.6446,0.7459,0.6546,0.5675,0.6862,15.71%
Explanation Completeness,0.5230,0.5848,0.5348,0.4427,0.5207,11.81%
Claim Accuracy,0.6651,0.7613,0.6826,0.5674,0.7295,14.46%
Claim Support,1.4387,1.7139,1.6012,1.0484,1.6781,19.13%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5636,0.5518,0.4529,0.5215,0.5202,0.00%
BERTScore (Precision),0.5651,0.5623,0.4583,0.5294,0.5266,0.00%
BERTScore (Recall),0.5645,0.5438,0.4491,0.5165,0.5162,0.00%
Fluency,0.7707,0.7016,0.5602,0.6515,0.6477,0.00%
Coherence,0.7736,0.7759,0.7623,0.8007,0.8235,6.45%
Explanation Accuracy,0.7877,0.7732,0.6372,0.7244,0.7334,0.00%
Explanation Completeness,0.6285,0.5969,0.4813,0.5705,0.5678,0.00%
Claim Accuracy,0.8065,0.8003,0.6449,0.7526,0.7496,0.00%
Claim Support,1.8456,1.2875,1.5307,1.6860,1.6544,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5253,0.5728,0.4570,0.5117,0.5506,9.03%
BERTScore (Precision),0.5245,0.5840,0.4670,0.5210,0.5586,11.35%
BERTScore (Recall),0.5282,0.5639,0.4508,0.5042,0.5446,6.76%
Fluency,0.7113,0.7118,0.5394,0.6353,0.7265,2.13%
Coherence,0.8330,0.9683,0.7027,0.6972,0.7952,16.24%
Explanation Accuracy,0.7242,0.7997,0.6284,0.7083,0.7700,10.42%
Explanation Completeness,0.5849,0.6172,0.4992,0.5594,0.5931,5.52%
Claim Accuracy,0.7573,0.8254,0.6611,0.7372,0.7962,8.98%
Claim Support,1.9089,2.0669,1.2746,1.7211,1.8268,8.28%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6109,0.1391,0.3584,0.2482,0.2401,0.00%
BERTScore (Precision),0.6144,0.1445,0.3729,0.2632,0.2511,0.00%
BERTScore (Recall),0.6093,0.1344,0.3477,0.2363,0.2311,0.00%
Fluency,0.7543,0.1654,0.4896,0.2684,0.3291,0.00%
Coherence,0.8330,0.2485,0.6556,0.4173,0.4235,0.00%
Explanation Accuracy,0.8509,0.1500,0.3973,0.2928,0.2077,0.00%
Explanation Completeness,0.6418,0.1129,0.3177,0.2275,0.1693,0.00%
Claim Accuracy,0.8708,0.1351,0.3882,0.2688,0.2149,0.00%
Claim Support,1.8791,0.2977,0.7210,0.6000,0.5355,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6236,0.4474,0.5170,0.3977,0.5012,0.00%
BERTScore (Precision),0.6521,0.4785,0.5462,0.4246,0.5245,0.00%
BERTScore (Recall),0.6000,0.4231,0.4940,0.3772,0.4829,0.00%
Fluency,0.7016,0.3693,0.5147,0.4433,0.5971,0.00%
Coherence,0.9855,0.7655,0.8141,0.5955,0.7577,0.00%
Explanation Accuracy,0.8421,0.5994,0.6893,0.5140,0.6128,0.00%
Explanation Completeness,0.5911,0.4313,0.4826,0.3555,0.4485,0.00%
Claim Accuracy,0.8758,0.6424,0.7139,0.5206,0.6319,0.00%
Claim Support,1.8868,1.2335,1.5065,1.0310,1.4629,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.4935,0.5067,0.4236,0.3938,0.4179,2.68%
BERTScore (Precision),0.4363,0.4392,0.3742,0.3484,0.3775,0.65%
BERTScore (Recall),0.5713,0.6043,0.4929,0.4565,0.4775,5.77%
Fluency,0.6717,0.6451,0.6323,0.4621,0.5777,0.00%
Coherence,0.6200,0.7873,0.7765,0.5049,0.7234,26.97%
Explanation Accuracy,0.6499,0.6330,0.3720,0.4896,0.3800,0.00%
Explanation Completeness,0.6156,0.6475,0.4147,0.4894,0.3959,5.19%
Claim Accuracy,0.6395,0.6550,0.3942,0.5002,0.3861,2.44%
Claim Support,0.2991,1.2760,1.1869,0.6931,1.2834,329.05%


# Claude 3 Haiku Tables

In [7]:
print("Processing Claude3_haiku")
process_llm(base_path, 'Anthropic/claude3_haiku', scenarios, agent_types)

Processing Claude3_haiku
Results for CLAUDE3_HAIKU


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5969,0.5638,0.5736,0.5292,0.5655,0.00%
BERTScore (Precision),0.5952,0.5764,0.5887,0.5346,0.5727,0.00%
BERTScore (Recall),0.6021,0.5555,0.5631,0.5261,0.5609,0.00%
Fluency,0.8480,0.7215,0.7225,0.7064,0.7836,0.00%
Coherence,0.9122,0.9207,0.7578,0.7427,0.8602,0.93%
Explanation Accuracy,0.8295,0.7955,0.7850,0.7046,0.7976,0.00%
Explanation Completeness,0.6675,0.6152,0.6149,0.5565,0.6214,0.00%
Claim Accuracy,0.8487,0.8203,0.8074,0.7444,0.8263,0.00%
Claim Support,1.7335,1.2807,1.4933,1.1934,1.7320,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5985,0.5401,0.5581,0.5442,0.5638,0.00%
BERTScore (Precision),0.5928,0.5543,0.5773,0.5490,0.5696,0.00%
BERTScore (Recall),0.6077,0.5346,0.5440,0.5416,0.5612,0.00%
Fluency,0.8685,0.7155,0.6572,0.7488,0.8098,0.00%
Coherence,0.8866,0.7442,0.8702,0.8694,0.8231,0.00%
Explanation Accuracy,0.8264,0.7411,0.7566,0.7605,0.7954,0.00%
Explanation Completeness,0.6624,0.5925,0.6038,0.5950,0.6180,0.00%
Claim Accuracy,0.8458,0.7910,0.7907,0.7828,0.8184,0.00%
Claim Support,1.4123,1.6939,1.7215,1.5063,1.5319,21.89%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5968,0.5416,0.5717,0.5228,0.5253,0.00%
BERTScore (Precision),0.5977,0.5467,0.5783,0.5367,0.5272,0.00%
BERTScore (Recall),0.6004,0.5402,0.5683,0.5131,0.5268,0.00%
Fluency,0.8320,0.7198,0.7712,0.6765,0.7618,0.00%
Coherence,0.8815,0.8087,0.9014,0.8050,0.7512,2.26%
Explanation Accuracy,0.8345,0.7669,0.7775,0.7448,0.7372,0.00%
Explanation Completeness,0.6674,0.5899,0.6047,0.5685,0.5799,0.00%
Claim Accuracy,0.8626,0.8045,0.8164,0.7684,0.7548,0.00%
Claim Support,1.5069,1.5192,1.8444,1.1814,1.1976,22.39%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6004,0.5352,0.5797,0.4825,0.5369,0.00%
BERTScore (Precision),0.6001,0.5467,0.5897,0.4925,0.5419,0.00%
BERTScore (Recall),0.6045,0.5276,0.5734,0.4762,0.5357,0.00%
Fluency,0.8654,0.7286,0.8151,0.6410,0.7192,0.00%
Coherence,0.8523,0.6442,0.8706,0.7986,0.8119,2.15%
Explanation Accuracy,0.8270,0.7620,0.7782,0.6896,0.7385,0.00%
Explanation Completeness,0.6681,0.5758,0.6185,0.5183,0.5855,0.00%
Claim Accuracy,0.8509,0.7794,0.7975,0.7330,0.7759,0.00%
Claim Support,1.6753,1.4484,1.8056,1.1069,1.2472,7.78%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6103,0.2089,0.3078,0.2531,0.2052,0.00%
BERTScore (Precision),0.5922,0.2183,0.3346,0.2641,0.2182,0.00%
BERTScore (Recall),0.6325,0.2007,0.2859,0.2438,0.1943,0.00%
Fluency,0.8459,0.3570,0.5307,0.4529,0.3802,0.00%
Coherence,0.8833,0.4828,0.6613,0.4071,0.3430,0.00%
Explanation Accuracy,0.8170,0.0965,0.0426,0.1191,0.0515,0.00%
Explanation Completeness,0.6625,0.0918,0.0766,0.1135,0.0674,0.00%
Claim Accuracy,0.8058,0.0865,0.0297,0.1142,0.0498,0.00%
Claim Support,1.7769,0.3686,0.4469,0.4312,0.3966,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.6249,0.5011,0.4843,0.3879,0.4849,0.00%
BERTScore (Precision),0.6288,0.5247,0.5120,0.4087,0.5035,0.00%
BERTScore (Recall),0.6232,0.4820,0.4616,0.3715,0.4699,0.00%
Fluency,0.8551,0.6551,0.6071,0.4667,0.6350,0.00%
Coherence,0.9025,0.8695,0.8295,0.5926,0.8143,0.00%
Explanation Accuracy,0.8579,0.5879,0.5985,0.5097,0.5982,0.00%
Explanation Completeness,0.6090,0.4394,0.4443,0.3798,0.4433,0.00%
Claim Accuracy,0.8636,0.5895,0.6156,0.5118,0.6138,0.00%
Claim Support,1.4435,1.1899,1.3424,0.9711,1.0539,0.00%


Agent,Base,EXP,COH,Base+EXP,Base+EXP+COH,Relative Gain
Metric,,,,,,
BERTScore (F1),0.5933,0.4998,0.4322,0.4730,0.4829,0.00%
BERTScore (Precision),0.5109,0.4356,0.3801,0.4144,0.4196,0.00%
BERTScore (Recall),0.7110,0.5998,0.5086,0.5585,0.5778,0.00%
Fluency,0.8561,0.7205,0.6415,0.6584,0.6798,0.00%
Coherence,0.7059,0.8440,0.7346,0.7301,0.7939,19.57%
Explanation Accuracy,0.8096,0.5690,0.4271,0.6010,0.5870,0.00%
Explanation Completeness,0.7732,0.5481,0.4239,0.5712,0.5643,0.00%
Claim Accuracy,0.8209,0.5759,0.4402,0.6183,0.6083,0.00%
Claim Support,0.4158,0.9025,0.7090,0.8753,0.9197,121.19%
